In [ ]:
import os
import requests
import json

HUBSPOT_BASE_URL = "https://api.hubapi.com"
HUBSPOT_API_KEY = os.environ.get('HUBSPOT_API_KEY')

AI21LABS_BASE_URL = "https://api.ai21.com/studio/v1/j2-ultra/chat"
AI21LABS_API_KEY = os.environ.get('AI21LABS_API_KEY')


HUBSPOT_HEADER = {'Authorization': f'Bearer {HUBSPOT_API_KEY}',
          'Content-Type': 'application/json'}

AI21LABS_HEADER = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": f'Bearer {AI21LABS_API_KEY}'
}

In [ ]:
def get_cursor_token(response): 
    return response.json()["paging"]["next"]["after"] if "paging" in response.json() and "next" in response.json()["paging"] else None

In [ ]:
pages = []

def get_all_pages():
    try:
        cursor_token = None

        response = requests.get(f'{HUBSPOT_BASE_URL}/cms/v3/pages/site-pages', headers=HUBSPOT_HEADER)
        response.raise_for_status()
        
        pages.extend(response.json()["results"])
        print(f"Number of pages: {len(pages)}")

        cursor_token = get_cursor_token(response)
        
        while cursor_token:
            r = requests.get(f"{HUBSPOT_BASE_URL}/cms/v3/pages/site-pages?after={cursor_token}", headers=HUBSPOT_HEADER)
            r.raise_for_status()
            pages.extend(r.json()["results"])

            cursor_token = get_cursor_token(r)
            print(f"Number of pages: {len(pages)}")


    except requests.exceptions.RequestException as e:
        print(f"Request error - {e}")

def get_cursor_token(response): 
    return response.json()["paging"]["next"]["after"] if "paging" in response.json() and "next" in response.json()["paging"] else None

In [ ]:
get_all_pages()

In [ ]:
def create_prompt(url):
    return f"Create a meta description for the following page: '{url}' on the website of the software company Metal Toad. Maximum 160 characters."

In [ ]:
def add_meta_description ():
    for page in pages:
        print(page["name"])
        print(page["url"])

        prompt = create_prompt(page['url'])
        payload = {
            "numResults": 1,
            "temperature": 0.7,
            "messages": [
                {"text": prompt, "role": "user"}
            ],
            "system": "You are an AI assistant for business research. Your responses should be informative and concise."
        }
        AI21_response = requests.post(AI21LABS_BASE_URL, json=payload, headers=AI21LABS_HEADER)
        AI21_response.raise_for_status()
        print(metaDescription)
        metaDescription = AI21_response.json()["outputs"][0]["text"]
        data = json.dumps({"metaDescription": metaDescription})
        
        hubspot_url = f"{HUBSPOT_BASE_URL}/{page['id']}"
        response = requests.patch(hubspot_url, data=data, headers=HUBSPOT_HEADER)
        response.raise_for_status()